In [85]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import missingno as msno
import seaborn as sns

In [86]:
path = "./data/"
books_df = pd.read_csv(path + "books.csv")
users_df = pd.read_csv(path + "users.csv")
rating_df = pd.read_csv(path + "train_ratings.csv")

In [87]:
user_rating_df = pd.merge(users_df, rating_df, how='left', on='user_id')
data = pd.merge(user_rating_df, books_df, how='left', on='isbn')
data.head(3)

,user_id,location,age,isbn,rating,book_title,book_author,year_of_publication,publisher,img_url,language,category,summary,img_path
0,8,"timmins, ontario, canada",NaN,0002005018,4.0,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,en,['Actresses'],"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg
1,8,"timmins, ontario, canada",NaN,074322678X,4.0,Where You'll Find Me: And Other Stories,Ann Beattie,2002.0,Scribner,http://images.amazon.com/images/P/074322678X.0...,en,['Fiction'],"Now back in print, Ann Beattie&#39;s finest sh...",images/074322678X.01.THUMBZZZ.jpg
2,8,"timmins, ontario, canada",NaN,0887841740,2.0,The Middle Stories,Sheila Heti,2004.0,House of Anansi Press,http://images.amazon.com/images/P/0887841740.0...,NaN,NaN,NaN,images/0887841740.01.THUMBZZZ.jpg


In [88]:
print(len(data))
data['user_id'].nunique()

315084


68092

In [89]:
data['location'] = data['location'].str.replace(r'[^0-9a-zA-Z:,]', '') # 특수문자 제거

data['location_city'] = data['location'].apply(lambda x: x.split(',')[0].strip())
data['location_state'] = data['location'].apply(lambda x: x.split(',')[1].strip())
data['location_country'] = data['location'].apply(lambda x: x.split(',')[2].strip())

data = data.replace('na', np.nan) #특수문자 제거로 n/a가 na로 바뀌게 되었습니다. 따라서 이를 컴퓨터가 인식할 수 있는 결측값으로 변환합니다.
data = data.replace('', np.nan) # 일부 경우 , , ,으로 입력된 경우가 있었으므로 이런 경우에도 결측값으로 변환합니다.

data.drop('location', axis=1, inplace=True)

/tmp/ipykernel_8888/322630745.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data['location'] = data['location'].str.replace(r'[^0-9a-zA-Z:,]', '') # 특수문자 제거


In [90]:
data.drop(columns = ["img_path","img_url"], inplace=True)

In [131]:
def fillAge(idx):
    global data
    d2 = data.drop(idx, axis=0)
    d2 = d2.groupby('isbn').aggregate({"age":np.average})
    d2['age'] = d2['age'].astype(int)
    
    avg_age = {}
    for i, isbn in data.loc[idx,'isbn'].items():
        try: 
            avg_age[i] = d2.loc[isbn, 'age']
        except:
            pass
        
    avg_age = pd.DataFrame(avg_age, index=[0]).T
    avg_age.rename(columns={0:"avgAge"}, inplace=True)
    return avg_age

In [132]:
age_na_idx = data[data['age'].isna()].index

In [133]:
a = fillAge(age_na_idx)
a

,avgAge
0,39
19,31
20,41
21,28
22,36
...,...
314282,36
314380,16
314402,41
314452,28


In [10]:
data.loc[a.index, 'age'] = a['avgAge'].apply(lambda x: x)
data.loc[age_na_idx].head(5)

,user_id,age,isbn,rating,book_title,book_author,year_of_publication,publisher,language,category,summary,location_city,location_state,location_country
0,8,39.0,0002005018,4.0,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,en,['Actresses'],"In a small town in Canada, Clara Callan reluct...",timmins,ontario,canada
1,8,NaN,074322678X,4.0,Where You'll Find Me: And Other Stories,Ann Beattie,2002.0,Scribner,en,['Fiction'],"Now back in print, Ann Beattie&#39;s finest sh...",timmins,ontario,canada
2,8,NaN,0887841740,2.0,The Middle Stories,Sheila Heti,2004.0,House of Anansi Press,NaN,NaN,NaN,timmins,ontario,canada
3,8,NaN,1552041778,2.0,Jane Doe,R. J. Kaiser,1999.0,Mira Books,NaN,NaN,NaN,timmins,ontario,canada
4,8,NaN,1567407781,6.0,The Witchfinder (Amos Walker Mystery Series),Loren D. Estleman,1998.0,Brilliance Audio - Trade,NaN,NaN,NaN,timmins,ontario,canada


In [37]:
bcpy = books_df.copy(deep=True)

In [38]:
import re
bcpy.loc[bcpy[bcpy['category'].notnull()].index, 'category'] = bcpy[bcpy['category'].notnull()]['category'].apply(lambda x: re.sub('[\W_]+',' ',x).strip())

In [39]:
bcpy['category'] = bcpy['category'].str.lower()

In [40]:
category_df = pd.DataFrame(bcpy['category'].value_counts()).reset_index()
category_df.columns = ['category','count']
category_df.head()

,category,count
0,fiction,33016
1,juvenile fiction,5835
2,biography autobiography,3326
3,history,1927
4,religion,1818


In [41]:
category_df2 = category_df[category_df['count']>=50]
categories = category_df2['category']

In [42]:
for category in categories:
    bcpy.loc[bcpy[bcpy['category'].str.contains(category,na=False)].index,'category_high'] = category

In [43]:
category_high_df = pd.DataFrame(bcpy['category_high'].value_counts()).reset_index()
category_high_df.columns = ['category','count']

In [44]:
bcpy['category_high'].fillna('others', inplace=True)

In [45]:
bcpy['category_high'].isna().sum()

0

In [46]:
bcpy.drop(columns=['category'], inplace=True)
bcpy.rename(columns={'category_high':'category'}, inplace=True)

In [47]:
bcpy['category'].value_counts()

others                       77677
fiction                      33410
juvenile fiction              5837
biography autobiography       3326
history                       1949
                             ...  
great britain                   56
authors american                53
fairy tales                     53
german fiction                  52
adventure and adventurers       51
Name: category, Length: 74, dtype: int64

In [48]:
bcpy

,isbn,book_title,book_author,year_of_publication,publisher,img_url,language,summary,img_path,category
0,0002005018,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,en,"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg,others
1,0060973129,Decision in Normandy,Carlo D'Este,1991.0,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,en,"Here, for the first time in paperback, is an o...",images/0060973129.01.THUMBZZZ.jpg,others
2,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,en,"Describes the great flu epidemic of 1918, an o...",images/0374157065.01.THUMBZZZ.jpg,medical
3,0399135782,The Kitchen God's Wife,Amy Tan,1991.0,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,en,A Chinese immigrant who is convinced she is dy...,images/0399135782.01.THUMBZZZ.jpg,fiction
4,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000.0,Berkley Publishing Group,http://images.amazon.com/images/P/0425176428.0...,en,"Essays by respected military historians, inclu...",images/0425176428.01.THUMBZZZ.jpg,history
...,...,...,...,...,...,...,...,...,...,...
149565,067161746X,The Bachelor Home Companion: A Practical Guide...,P.J. O'Rourke,1987.0,Pocket Books,http://images.amazon.com/images/P/067161746X.0...,en,A tongue-in-cheek survival guide for single pe...,images/067161746X.01.THUMBZZZ.jpg,humor
149566,0767907566,All Elevations Unknown: An Adventure in the He...,Sam Lightner,2001.0,Broadway Books,http://images.amazon.com/images/P/0767907566.0...,en,A daring twist on the travel-adventure genre t...,images/0767907566.01.THUMBZZZ.jpg,nature
149567,0884159221,Why stop?: A guide to Texas historical roadsid...,Claude Dooley,1985.0,Lone Star Books,http://images.amazon.com/images/P/0884159221.0...,NaN,NaN,images/0884159221.01.THUMBZZZ.jpg,others
149568,0912333022,The Are You Being Served? Stories: 'Camping In...,Jeremy Lloyd,1997.0,Kqed Books,http://images.amazon.com/images/P/0912333022.0...,en,These hilarious stories by the creator of publ...,images/0912333022.01.THUMBZZZ.jpg,fiction


user 데이터 age 처리한거로 바꾸기..

In [24]:
ucpy = users_df.copy(deep=True)
t = data[['user_id','age']]
idx = t['age'].dropna().index
t = t.loc[idx]
t = t.groupby('user_id').aggregate({"age":np.max}).squeeze()
ucpy['user_id2'] = ucpy['user_id'].copy()
a = ucpy[['user_id', 'user_id2']].set_index('user_id2').squeeze()
a = a.map(t).reset_index()
a.columns = ['user_id','age']
ucpy['age'] = a['age']
ucpy.drop(columns = 'user_id2', inplace=True)
ucpy

,user_id,location,age
0,8,"timmins, ontario, canada",39.0
1,11400,"ottawa, ontario, canada",49.0
2,11676,"n/a, n/a, n/a",90.0
3,67544,"toronto, ontario, canada",30.0
4,85526,"victoria, british columbia, canada",36.0
...,...,...,...
68087,278376,"danville, pennsylvania, usa",54.0
68088,278621,"victoria, delaware, canada",74.0
68089,278636,"irvington, alabama, usa",NaN
68090,278659,"vancouver, washington, usa",33.0


In [49]:
books_df

,isbn,book_title,book_author,year_of_publication,publisher,img_url,language,category,summary,img_path
0,0002005018,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,en,['Actresses'],"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg
1,0060973129,Decision in Normandy,Carlo D'Este,1991.0,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,en,['1940-1949'],"Here, for the first time in paperback, is an o...",images/0060973129.01.THUMBZZZ.jpg
2,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,en,['Medical'],"Describes the great flu epidemic of 1918, an o...",images/0374157065.01.THUMBZZZ.jpg
3,0399135782,The Kitchen God's Wife,Amy Tan,1991.0,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,en,['Fiction'],A Chinese immigrant who is convinced she is dy...,images/0399135782.01.THUMBZZZ.jpg
4,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000.0,Berkley Publishing Group,http://images.amazon.com/images/P/0425176428.0...,en,['History'],"Essays by respected military historians, inclu...",images/0425176428.01.THUMBZZZ.jpg
...,...,...,...,...,...,...,...,...,...,...
149565,067161746X,The Bachelor Home Companion: A Practical Guide...,P.J. O'Rourke,1987.0,Pocket Books,http://images.amazon.com/images/P/067161746X.0...,en,['Humor'],A tongue-in-cheek survival guide for single pe...,images/067161746X.01.THUMBZZZ.jpg
149566,0767907566,All Elevations Unknown: An Adventure in the He...,Sam Lightner,2001.0,Broadway Books,http://images.amazon.com/images/P/0767907566.0...,en,['Nature'],A daring twist on the travel-adventure genre t...,images/0767907566.01.THUMBZZZ.jpg
149567,0884159221,Why stop?: A guide to Texas historical roadsid...,Claude Dooley,1985.0,Lone Star Books,http://images.amazon.com/images/P/0884159221.0...,NaN,NaN,NaN,images/0884159221.01.THUMBZZZ.jpg
149568,0912333022,The Are You Being Served? Stories: 'Camping In...,Jeremy Lloyd,1997.0,Kqed Books,http://images.amazon.com/images/P/0912333022.0...,en,['Fiction'],These hilarious stories by the creator of publ...,images/0912333022.01.THUMBZZZ.jpg


In [50]:
bcpy

,isbn,book_title,book_author,year_of_publication,publisher,img_url,language,summary,img_path,category
0,0002005018,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,en,"In a small town in Canada, Clara Callan reluct...",images/0002005018.01.THUMBZZZ.jpg,others
1,0060973129,Decision in Normandy,Carlo D'Este,1991.0,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,en,"Here, for the first time in paperback, is an o...",images/0060973129.01.THUMBZZZ.jpg,others
2,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,en,"Describes the great flu epidemic of 1918, an o...",images/0374157065.01.THUMBZZZ.jpg,medical
3,0399135782,The Kitchen God's Wife,Amy Tan,1991.0,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,en,A Chinese immigrant who is convinced she is dy...,images/0399135782.01.THUMBZZZ.jpg,fiction
4,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000.0,Berkley Publishing Group,http://images.amazon.com/images/P/0425176428.0...,en,"Essays by respected military historians, inclu...",images/0425176428.01.THUMBZZZ.jpg,history
...,...,...,...,...,...,...,...,...,...,...
149565,067161746X,The Bachelor Home Companion: A Practical Guide...,P.J. O'Rourke,1987.0,Pocket Books,http://images.amazon.com/images/P/067161746X.0...,en,A tongue-in-cheek survival guide for single pe...,images/067161746X.01.THUMBZZZ.jpg,humor
149566,0767907566,All Elevations Unknown: An Adventure in the He...,Sam Lightner,2001.0,Broadway Books,http://images.amazon.com/images/P/0767907566.0...,en,A daring twist on the travel-adventure genre t...,images/0767907566.01.THUMBZZZ.jpg,nature
149567,0884159221,Why stop?: A guide to Texas historical roadsid...,Claude Dooley,1985.0,Lone Star Books,http://images.amazon.com/images/P/0884159221.0...,NaN,NaN,images/0884159221.01.THUMBZZZ.jpg,others
149568,0912333022,The Are You Being Served? Stories: 'Camping In...,Jeremy Lloyd,1997.0,Kqed Books,http://images.amazon.com/images/P/0912333022.0...,en,These hilarious stories by the creator of publ...,images/0912333022.01.THUMBZZZ.jpg,fiction


In [67]:
path = "./testdata/"
ucpy.to_csv(path + 'users.csv', index=False)
bcpy.to_csv(path + 'books.csv', index=False)

In [54]:
import os
os.getcwd()

'/opt/ml/input/code'

In [68]:
!python3 main.py --DATA_PATH "./testdata/" --MODEL FM

--------------- FM Load Data ---------------
--------------- FM Train/Valid Split ---------------
--------------- INIT FM ---------------
--------------- FM TRAINING ---------------
100%|███████████████████████████████████████████| 60/60 [00:00<00:00, 87.73it/s]
epoch: 0 validation: rmse: 2.7679833033892027
100%|██████████████████████████████████████████| 60/60 [00:00<00:00, 104.75it/s]
epoch: 1 validation: rmse: 2.475991404585923
100%|███████████████████████████████████████████| 60/60 [00:00<00:00, 81.34it/s]
epoch: 2 validation: rmse: 2.3829356059129907
100%|██████████████████████████████████████████| 60/60 [00:00<00:00, 105.92it/s]
epoch: 3 validation: rmse: 2.3458496143260077
100%|██████████████████████████████████████████| 60/60 [00:00<00:00, 106.52it/s]
epoch: 4 validation: rmse: 2.3367826951800854
100%|██████████████████████████████████████████| 60/60 [00:00<00:00, 107.23it/s]
epoch: 5 validation: rmse: 2.3405081730325192
100%|██████████████████████████████████████████| 60/60 [0

In [78]:
!python3 main.py --DATA_PATH "./testdata/" --MODEL FFM

--------------- FFM Load Data ---------------
--------------- FFM Train/Valid Split ---------------
--------------- INIT FFM ---------------
--------------- FFM TRAINING ---------------
100%|███████████████████████████████████████████| 60/60 [00:00<00:00, 83.05it/s]
epoch: 0 validation: rmse: 2.715202310271589
100%|███████████████████████████████████████████| 60/60 [00:00<00:00, 92.10it/s]
epoch: 1 validation: rmse: 2.412607938435639
100%|███████████████████████████████████████████| 60/60 [00:00<00:00, 91.86it/s]
epoch: 2 validation: rmse: 2.3919471399178103
100%|███████████████████████████████████████████| 60/60 [00:00<00:00, 82.92it/s]
epoch: 3 validation: rmse: 2.396635256222482
100%|███████████████████████████████████████████| 60/60 [00:00<00:00, 91.77it/s]
epoch: 4 validation: rmse: 2.409422353229036
100%|███████████████████████████████████████████| 60/60 [00:00<00:00, 81.65it/s]
epoch: 5 validation: rmse: 2.406661415753758
100%|███████████████████████████████████████████| 60/60 [

In [76]:
!python3 main.py --DATA_PATH "./testdata/" --MODEL DeepCoNN

--------------- DeepCoNN Load Data ---------------
[nltk_data] Downloading package punkt to /opt/ml/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification

In [134]:
!python3 main.py -h

usage: main.py [-h] [--DATA_PATH DATA_PATH]
               [--MODEL {FM,FFM,NCF,WDN,DCN,CNN_FM,DeepCoNN}]
               [--DATA_SHUFFLE DATA_SHUFFLE] [--TEST_SIZE TEST_SIZE]
               [--SEED SEED] [--BATCH_SIZE BATCH_SIZE] [--EPOCHS EPOCHS]
               [--LR LR] [--WEIGHT_DECAY WEIGHT_DECAY] [--DEVICE {cuda,cpu}]
               [--FM_EMBED_DIM FM_EMBED_DIM] [--FFM_EMBED_DIM FFM_EMBED_DIM]
               [--NCF_EMBED_DIM NCF_EMBED_DIM] [--NCF_MLP_DIMS NCF_MLP_DIMS]
               [--NCF_DROPOUT NCF_DROPOUT] [--WDN_EMBED_DIM WDN_EMBED_DIM]
               [--WDN_MLP_DIMS WDN_MLP_DIMS] [--WDN_DROPOUT WDN_DROPOUT]
               [--DCN_EMBED_DIM DCN_EMBED_DIM] [--DCN_MLP_DIMS DCN_MLP_DIMS]
               [--DCN_DROPOUT DCN_DROPOUT] [--DCN_NUM_LAYERS DCN_NUM_LAYERS]
               [--CNN_FM_EMBED_DIM CNN_FM_EMBED_DIM]
               [--CNN_FM_LATENT_DIM CNN_FM_LATENT_DIM]
               [--DEEPCONN_VECTOR_CREATE DEEPCONN_VECTOR_CREATE]
               [--DEEPCONN_EMBED_DIM DEEPCONN_

In [84]:
!python3 ensemble.py --ENSEMBLE_FILES 20221027_152933_FM,20221027_170634_FFM,20221027_170448_DeepCoNN --ENSEMBLE_STRATEGY WEIGHTED --ENSEMBLE_WEIGHT 0.35,0.35,0.3